In [2]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

In [9]:
class tempC_NETCDF4:
    pathname = "http://thredds.northwestknowledge.net:8080/thredds/dodsC/NWCSC_INTEGRATED_SCENARIOS_ALL_CLIMATE/bcsd-nmme/rawForecasts/nmme_hindcast_tmean.nc"
    lat_target=31.105
    lon_target=-97.35
    time_num=504
    time_index=range(0,time_num,1)
    filehandle=Dataset(pathname,'r',format="NETCDF4")
    handles = {'tCFSv2' : filehandle.variables['tmean_CFSv2'],
           'tCMC1' : filehandle.variables['tmean_CMC1'],
           'tCMC2' : filehandle.variables['tmean_CMC2'],
           'tGFDL' : filehandle.variables['tmean_GFDL'],
           'tGFDL_FLOR' : filehandle.variables['tmean_GFDL_FLOR'],
           'tNASA' : filehandle.variables['tmean_NASA'],
           'tNCAR' : filehandle.variables['tmean_NCAR'],
           'tENSMEAN' : filehandle.variables['tmean_ENSMEAN'],
           'tCanCM4i' : filehandle.variables['tmean_CanCM4i'],
           'tGEM_NEMO' : filehandle.variables['tmean_GEM_NEMO'],
           'tGFDL_SPEAR' : filehandle.variables['tmean_GFDL_SPEAR'],
           'tNCAR_CESM1' : filehandle.variables['tmean_NCAR_CESM1'],
           'lat': filehandle.variables['lat'],
           'lon': filehandle.variables['lon'],
           'time': filehandle.variables['time']}
    
    def get_models(self, handles):
        keys = handles.keys()
        models = []
        for key in keys:
            models.append(key)
        models.remove('lat')
        models.remove('lon')
        models.remove('time')
        return models
    
    def get_lat(self, handles, lat_target):
        lat = handles['lat'][:]
        lat_index =   (np.abs(lat-lat_target)).argmin() 
        if(lat[lat_index]>lat_target):
            if(lat_index!=0):
                lat_index = lat_index - 1
        if(lat[lat_index]<lat_target):
            if(lat_index!=len(lat)):
                lat_index =lat_index +1
        return lat_index
    
    def get_lon(self, handles, lon_target):
        lon = handles['lon'][:]
        lon_index =   (np.abs(lon-lon_target)).argmin()
        if(lon[lon_index]>lon_target):
            if(lon_index!=0):
                lon_index = lon_index - 1
        if(lon[lon_index]<lon_target):
            if(lon_index!=len(lon)):
                lon_index = lon_index + 1
        return lon_index

    def dates(self):
        begin_date = datetime.date(1982, 1, 1)
        end_date = datetime.date(2023, 12, 31)
        dates = []
        while begin_date <= end_date:
            dates.append(begin_date)
            begin_date += relativedelta(months = 1)
        return dates
    
    def get_data(self, models, time_index, lat_index, lon_index, l):
        model_dic = {}
        for model in models:
            data = tempC_NETCDF4.handles[model][:, time_index, lat_index,
                                                 lon_index]
            monthly_c = []
            for count in range(len(data[l])):
                deg_c = ((data[0][count] - 273.15))
                deg_c = deg_c.round(2)
                monthly_c.append(deg_c)
            model_dic[model] = monthly_c
        return model_dic

In [22]:
tempC = tempC_NETCDF4()
models = tempC.get_models(tempC.handles)
lat_index = tempC.get_lat(tempC.handles, tempC.lat_target)
lon_index = tempC.get_lon(tempC.handles, tempC.lon_target)
dates = tempC.dates()
df0 = pd.DataFrame(data = tempC.get_data(
    models, tempC.time_index, lat_index, lon_index, 0), index = dates)
df1 = pd.DataFrame(data = tempC.get_data(
    models, tempC.time_index, lat_index, lon_index, 1), index = dates)
df2 = pd.DataFrame(data = tempC.get_data(
    models, tempC.time_index, lat_index, lon_index, 2), index = dates)
df3 = pd.DataFrame(data = tempC.get_data(
    models, tempC.time_index, lat_index, lon_index, 3), index = dates)
df4 = pd.DataFrame(data = tempC.get_data(
    models, tempC.time_index, lat_index, lon_index, 4), index = dates)
df5 = pd.DataFrame(data = tempC.get_data(
    models, tempC.time_index, lat_index, lon_index, 5), index = dates)
df6 = pd.DataFrame(data = tempC.get_data(
    models, tempC.time_index, lat_index, lon_index, 6), index = dates)
df7 = pd.DataFrame(data = tempC.get_data(
    models, tempC.time_index, lat_index, lon_index, 7), index = dates)
df8 = pd.DataFrame(data = tempC.get_data(
    models, tempC.time_index, lat_index, lon_index, 8), index = dates)

In [25]:
with pd.ExcelWriter('Monthly_temp_C.xlsx') as writer:
    df0.to_excel(writer, sheet_name = 'lead 1')
    df1.to_excel(writer, sheet_name = 'lead 2')
    df2.to_excel(writer, sheet_name = 'lead 3')
    df3.to_excel(writer, sheet_name = 'lead 4')
    df4.to_excel(writer, sheet_name = 'lead 5')
    df5.to_excel(writer, sheet_name = 'lead 6')
    df6.to_excel(writer, sheet_name = 'lead 7')
    df7.to_excel(writer, sheet_name = 'lead 8')
    df8.to_excel(writer, sheet_name = 'lead 9')